In [1]:
# Import dependencies 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dropout

# Load the dataset
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

/Users/yenori/anaconda3/envs/mydev/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/cz/_pyz_5qn69155q4_bc51whp00000gn/T/ipykernel_71420/1752333519.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-06 20:09:05.767681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX51

In [2]:

# Preprocessing steps (same as for initial model)
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = counts[counts < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = counts[counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
 
# Convert categorical data to numeric with `pd.get_dummies`
data_encoded = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
X = data_encoded.drop('IS_SUCCESSFUL', axis=1)
y = data_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [3]:
# Scale the features
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Optimize the model
nn_optimized = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.2),
    tf.keras.layers.Dense(50, activation='relu'),
    Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the optimized model
nn_optimized.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the optimized model
fit_model = nn_optimized.fit(X_train_scaled, y_train, epochs=100, batch_size=256, validation_split=0.2)


Epoch 1/100
81/81 [==============================] - 15s 56ms/step - loss: 0.6132 - accuracy: 0.6908 - val_loss: 0.5726 - val_accuracy: 0.7207
Epoch 2/100
81/81 [==============================] - 2s 23ms/step - loss: 0.5733 - accuracy: 0.7227 - val_loss: 0.5666 - val_accuracy: 0.7236
Epoch 3/100
81/81 [==============================] - 1s 18ms/step - loss: 0.5667 - accuracy: 0.7249 - val_loss: 0.5639 - val_accuracy: 0.7258
Epoch 4/100
81/81 [==============================] - 2s 26ms/step - loss: 0.5619 - accuracy: 0.7270 - val_loss: 0.5622 - val_accuracy: 0.7258
Epoch 5/100
81/81 [==============================] - 4s 44ms/step - loss: 0.5611 - accuracy: 0.7297 - val_loss: 0.5611 - val_accuracy: 0.7254
Epoch 6/100
81/81 [==============================] - 4s 46ms/step - loss: 0.5575 - accuracy: 0.7311 - val_loss: 0.5604 - val_accuracy: 0.7258
Epoch 7/100
81/81 [==============================] - 2s 28ms/step - loss: 0.5581 - accuracy: 0.7299 - val_loss: 0.5615 - val_accuracy: 0.7238
Epoch

In [4]:
# Evaluate the optimized model
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Model - Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the optimized model
nn_optimized.save('AlphabetSoupCharity_Optimization.h5')

268/268 - 1s - loss: 0.5543 - accuracy: 0.7249 - 904ms/epoch - 3ms/step
Optimized Model - Loss: 0.5543269515037537, Accuracy: 0.7248979806900024


/Users/yenori/anaconda3/envs/mydev/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
